In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import hamming 


## Read the master csv files and create the nearest neightbour function

In [50]:
df_item_ALL = pd.read_csv('./data/df_item_ALL.csv')

In [53]:
df_item_ALL

,ProductID,Name,Brand,Shop,Price,Sales_Number,Package,MadeIn,Description,Image,Review_avg,Review_total,recommendtofdratio,recommendtofd_num,Main_Cat,Sub_Cat
0,H29064,香濃黑芝麻高鈣低脂牛奶,安怡,安怡官方旗艦店,23.9,30000.0,180毫升 x 6,泰國,香濃黑芝麻高鈣低脂牛奶,https://images.hktvmall.com/h0888001/51c29a813...,4.3,293.0,0.87,34/39,Beverages,Dairy
1,U31804,[原箱] 微氣礦物質水 330毫升 X 24 #水 #sparkling #sparklin...,飛雪,太古可口可樂官方旗艦店,84.0,1000.0,330毫升 x24,香港,[原箱] 微氣礦物質水 330毫升 X 24 #水 #sparkling #sparklin...,https://images.hktvmall.com/h0888001/270113/h0...,4.9,61.0,1.0,48/48,Beverages,Water
2,S47915,[原箱] 無招紙礦物質水 600 毫升 X 15#無包裝 #行山 #環保,飛雪,太古可口可樂官方旗艦店,59.0,2000.0,600毫升 x 15,香港,[原箱] 無招紙礦物質水,https://images.hktvmall.com/h0888001/267592/h0...,4.9,74.0,1.0,48/48,Beverages,Water
3,J54903,津路,津路,零食大王 (零食飲品旗艦店),165.0,10000.0,500ml x24,中國,最受香港人喜愛嘅無糖茶茶香回甘 生津解渴去油膩感 阻擋油脂被腸道吸收,https://images.hktv-img.com/images/HKTV/10856/...,4.8,87.0,NaN,NaN,Beverages,No Sugar Tea
4,H22821,天然草本燃脂瘦身排毒花草茶28天早晚裝,ThinTea,澳聯商業有限公司,388.0,600.0,NaN,NaN,"ThinTea是從澳大利亞進口的瘦身減肥花草茶，以純天然有機花草茶配方提煉而成。-排清宿便,...",https://images.hktv-img.com/images/HKTV/19703/...,4.8,57.0,NaN,NaN,Beverages,No Sugar Tea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,O65031,[意大利直送] 純橄欖油 1L (平行進口),Farchioni.,山有店,78.0,6000.0,NaN,NaN,[意大利直送] 純橄欖油 1L (平行進口)傳承235年，家喻戶曉的義大利橄欖油品牌無摻假，...,https://images.hktv-img.com/images/HKTV/623/FC...,4.4,83.0,80%,12/15,Rice/Oil,Oil
4000,O20441,[意大利直送] 特級初榨橄欖油 1000ml (平行進口),Farchioni.,山有店,92.0,10000.0,NaN,NaN,[意大利直送] 特級初榨橄欖油 1000ml (平行進口)-意大利暢銷品牌-油貿質溫潤果香十...,https://images.hktv-img.com/images/HKTV/623/FC...,4.4,139.0,94%,35/37,Rice/Oil,Oil
4001,N07003,胚芽糙米 (一等米) 2.5公斤,皇家穀堡,山有店,68.0,9000.0,NaN,NaN,胚芽糙米 (一等米) 2.5公斤★純淨的花東滋味★不受汙染的新鮮純淨好米★調合糙米的營養與胚...,https://images.hktv-img.com/images/HKTV/623/TW...,4.4,133.0,89%,17/19,Rice/Oil,Rice
4002,M67892,特級冷壓初榨橄欖油 500ml (平行進口) [意大利直送],Farchioni.,山有店,58.0,10000.0,NaN,NaN,特級冷壓初榨橄欖油 500ml (平行進口) [意大利直送]-意大利暢銷品牌-油貿質溫潤果香...,https://images.hktv-img.com/images/HKTV/623/FC...,4.4,109.0,85%,12/14,Rice/Oil,Oil


In [66]:
# after trying to solve the 1d vector problem using the for loop,
# it is found that there are duplicated product ID
#therefore for products with duplicated rows, the df returns 2 rows
# solution? drop the duplicated rows
df_item_ALL.drop_duplicates(subset=['ProductID'],inplace=True)

In [67]:
# create a new dataframe with productID as index column. For retrieveing the info of the recommended products.
df_item_ALL_id_as_index = df_item_ALL.set_index('ProductID')

In [68]:
#keep only ProductID, brand, main_cat, and sub_cat for recommendation
# main_cat will be dropped later 
df_item_brand_cat = df_item_ALL[['ProductID','Main_Cat','Brand','Sub_Cat']]

In [69]:
#checking. NOT for streamlit
#df_item_brand_cat

In [70]:
# lets write a function to compute k nearest neighbours of active user
# To Find the k nearest neighbours of active user first find the distance of active user to all other users
def nearestneighbours_brand_cat(df,product,K):
    # create a user df that contains all users except active user
    allProducts = pd.DataFrame(df.index)
    allProducts = allProducts[allProducts.ProductID!=product]
    #Add a column to this df which contains distance of active user to each user
    allProducts["distance"] =allProducts["ProductID"].apply(lambda x: hamming(df.loc[product],df.loc[x]))        
    KnearestProducts = allProducts.sort_values(["distance"],ascending=True)["ProductID"][:K]
    return KnearestProducts

## create product with one-hotted brand and sub-cat, categorised by main cat

In [71]:
#list out all the main cats. NOT FOR streamlit
#df_item_ALL['Main_Cat'].unique()

In [72]:
#extract the data according to their main cat

#for brand and sub_cat
df_brand_cat_beverages = df_item_brand_cat[df_item_ALL['Main_Cat']=='Beverages'][['ProductID','Brand','Sub_Cat']]
df_brand_cat_frozen = df_item_brand_cat[df_item_ALL['Main_Cat']=='Frozen'][['ProductID','Brand','Sub_Cat']]
df_brand_cat_health = df_item_brand_cat[df_item_ALL['Main_Cat']=='Health'][['ProductID','Brand','Sub_Cat']]
df_brand_cat_skin_care = df_item_brand_cat[df_item_ALL['Main_Cat']=='Skin Care'][['ProductID','Brand','Sub_Cat']]
df_brand_cat_junk_food = df_item_brand_cat[df_item_ALL['Main_Cat']=='Junk Food'][['ProductID','Brand','Sub_Cat']]
df_brand_cat_noodles = df_item_brand_cat[df_item_ALL['Main_Cat']=='Noodles'][['ProductID','Brand','Sub_Cat']]
df_brand_cat_personal_care = df_item_brand_cat[df_item_ALL['Main_Cat']=='personal care'][['ProductID','Brand','Sub_Cat']]
df_brand_cat_rice_oil = df_item_brand_cat[df_item_ALL['Main_Cat']=='Rice/Oil'][['ProductID','Brand','Sub_Cat']]


In [73]:
#checking. NOT FOR streamlit
#df_brand_cat_junk_food 

In [74]:
#one-hot the brand and sub_cat
df_brand_cat_beverages = pd.concat([df_brand_cat_beverages['ProductID'],pd.get_dummies(df_brand_cat_beverages.Brand),pd.get_dummies(df_brand_cat_beverages.Sub_Cat)],axis=1)
df_brand_cat_frozen = pd.concat([df_brand_cat_frozen['ProductID'],pd.get_dummies(df_brand_cat_frozen.Brand),pd.get_dummies(df_brand_cat_frozen.Sub_Cat)],axis=1)
df_brand_cat_health = pd.concat([df_brand_cat_health['ProductID'],pd.get_dummies(df_brand_cat_health.Brand),pd.get_dummies(df_brand_cat_health.Sub_Cat)],axis=1)
df_brand_cat_skin_care  = pd.concat([df_brand_cat_skin_care['ProductID'],pd.get_dummies(df_brand_cat_skin_care.Brand),pd.get_dummies(df_brand_cat_skin_care.Sub_Cat)],axis=1)
df_brand_cat_junk_food = pd.concat([df_brand_cat_junk_food['ProductID'],pd.get_dummies(df_brand_cat_junk_food.Brand),pd.get_dummies(df_brand_cat_junk_food.Sub_Cat)],axis=1)
df_brand_cat_noodles = pd.concat([df_brand_cat_noodles['ProductID'],pd.get_dummies(df_brand_cat_noodles.Brand),pd.get_dummies(df_brand_cat_noodles.Sub_Cat)],axis=1)
df_brand_cat_personal_care = pd.concat([df_brand_cat_personal_care['ProductID'],pd.get_dummies(df_brand_cat_personal_care.Brand),pd.get_dummies(df_brand_cat_personal_care.Sub_Cat)],axis=1)
df_brand_cat_rice_oil = pd.concat([df_brand_cat_rice_oil['ProductID'],pd.get_dummies(df_brand_cat_rice_oil.Brand),pd.get_dummies(df_brand_cat_rice_oil.Sub_Cat)],axis=1)

In [75]:
#checking. NOT FOR streamlit
df_brand_cat_beverages

,ProductID,137 Degrees Nut Milk,2+3,5100.0,ARCHCAFE,Aplex,Arctic Power Berries,Awakening,Blue Diamond,B哥,...,No Sugar Tea,Plant Milk,Soft Drink,Soy Milk,Sparkling Water,Sweeten Tea,Tea Bag,Water,Wellness Drink,Yogurt Drink
0,H29064,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,U31804,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,S47915,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,J54903,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,H22821,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,C02997,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
716,D51094,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
717,C73339,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
718,B30291,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
#set ProductID as index column
df_brand_cat_beverages.set_index('ProductID',inplace=True)
df_brand_cat_frozen.set_index('ProductID',inplace=True)
df_brand_cat_health.set_index('ProductID',inplace=True)
df_brand_cat_skin_care.set_index('ProductID',inplace=True)
df_brand_cat_junk_food.set_index('ProductID',inplace=True)
df_brand_cat_noodles.set_index('ProductID',inplace=True)
df_brand_cat_personal_care.set_index('ProductID',inplace=True)
df_brand_cat_rice_oil.set_index('ProductID',inplace=True)

In [106]:
df_brand_cat_rice_oil

,Farchioni.,KINGS,刀嘜,獅球嘜,瑞茲米油組合,皇家穀堡,袋鼠米油組合,雙羊牌,Oil,Rice
ProductID,,,,,,,,,,
F80786,0,0,0,0,0,0,0,1,0,1
J09916,0,0,0,0,0,0,1,0,0,1
J19163,0,0,0,0,1,0,0,0,0,1
J18087,0,0,1,0,0,0,0,0,1,0
C56788,0,0,0,1,0,0,0,0,1,0
N99399,0,1,0,0,0,0,0,0,0,1
C56789,0,0,0,1,0,0,0,0,0,1
O65031,1,0,0,0,0,0,0,0,1,0
O20441,1,0,0,0,0,0,0,0,1,0


In [107]:
#testing the nearest neighbours function
recommend_items = nearestneighbours_brand_cat(df_brand_cat_rice_oil,'O65031',5)

In [108]:
recommend_items

8     O20441
10    M67892
3     J18087
4     C56788
0     F80786
Name: ProductID, dtype: object

In [109]:
for i in recommend_items.values:
    print(df_item_ALL_id_as_index.loc[i]['Name'])

[意大利直送] 特級初榨橄欖油 1000ml (平行進口)
特級冷壓初榨橄欖油 500ml (平行進口) [意大利直送]
純正花生油
花生油
澳洲雙羊百搭米王(非真空包裝) #雙羊牌#澳洲米#雙羊#米 


In [33]:
#df.loc[ID]
df_item_ALL_id_as_index.loc[ID]

,Name,Brand,Shop,Price,Sales_Number,Package,MadeIn,Description,Image,Review_avg,Review_total,recommendtofdratio,recommendtofd_num,Main_Cat,Sub_Cat
ProductID,,,,,,,,,,,,,,,
I91995,純黑咖啡 2g x 100,G7,TA 網上店,82.0,2000.0,NaN,NaN,越南中原的G7咖啡首屈一指，雖然是速溶咖啡，但是味道香濃醇厚。而每粒咖啡豆 (Buon Ma...,https://cdn-mms.hktvmall.com/hktv/mms/uploadPr...,4.7,70.0,1.0,2022-10-10 00:00:00,Beverages,Coffee
I91995,純黑咖啡 2g x 100,G7,TA 網上店,82.0,2000.0,NaN,NaN,越南中原的G7咖啡首屈一指，雖然是速溶咖啡，但是味道香濃醇厚。而每粒咖啡豆 (Buon Ma...,https://cdn-mms.hktvmall.com/hktv/mms/uploadPr...,4.7,70.0,1.0,2022-10-10 00:00:00,Beverages,Coffee


In [63]:
#allProducts["1st_vector"] = np.nan
allProducts["1st_vector"] =allProducts["ProductID"].apply(lambda x: len(df.loc[product]))
allProducts["2nd_vector"] =allProducts["ProductID"].apply(lambda x: np.atleast_1d(df.loc[x].squeeze()).ndim)

In [64]:
allProducts

,ProductID,1st_vector,2nd_vector
0,H29064,202,1
1,U31804,202,1
2,S47915,202,1
3,J54903,202,1
4,H22821,202,1
...,...,...,...
714,B07372,202,1
716,D51094,202,1
717,C73339,202,1
718,B30291,202,1
